<div>
<b><br><font face = "Gill Sans" size = "4"><center>Heart attack prediction with logistic regression</center></font></b>
</div>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [733 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,347 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubu

<span style="font-family:Gill Sans">
<h1>Hearth attach prediction with logisic regression</h1></span>

Let us use PySpark to predict if a person can suffer a heart attack based on some medical information.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

In [5]:
from pyspark.sql.types import StructField, StructType, FloatType, IntegerType

heart_prediction_schema = StructType([
    StructField("male", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("education", IntegerType(), True),
    StructField("currentSmoker", IntegerType(), True),
    StructField("cigsPerDay", IntegerType(), True),
    StructField("BPMeds", IntegerType(), True),
    StructField("prevalentStroke", IntegerType(), True),
    StructField("prevalentHyp", IntegerType(), True),
    StructField("diabetes", IntegerType(), True),
    StructField("totChol", FloatType(), True),
    StructField("sysBP", FloatType(), True),
    StructField("diaBP", FloatType(), True),
    StructField("BMI", FloatType(), True),
    StructField("heartRate", IntegerType(), True),
    StructField("glucose", IntegerType(), True),
    StructField("TenYearCHD", IntegerType(), True)
])

heart_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/drive/MyDrive/ColabNotebooks/BigDataSpring2024/datasets/heart_attack_prediction.csv").\
    schema(heart_prediction_schema).\
    load()
heart_df.printSchema()
heart_df.show(n=10, truncate=False)

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: float (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+----

In [6]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|      105|            0|        29|    53|              0|           0|       0|     50|    0|    0| 19|        1|    388|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [7]:
# Remove rows with at least one value as null


In [8]:
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|        0|            0|         0|     0|              0|           0|       0|      0|    0|    0|  0|        0|      0|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [9]:
# Assemble the features into a single vector column


In [12]:
# Generate predictions with the trained model

Coefficients: [0.06464168287662507,0.028948954746943024,0.0018687751872228128,0.01716158392333626,0.00626410609104656,0.011820670673763436,-0.009559295732016686,0.006744772352125628]
+--------------------+----------+----------+
|            features|TenYearCHD|prediction|
+--------------------+----------+----------+
|[34.0,0.0,226.0,1...|         0|       0.0|
|[34.0,0.0,196.0,1...|         0|       0.0|
|[35.0,0.0,170.0,9...|         0|       0.0|
|[35.0,5.0,186.0,1...|         0|       0.0|
|[35.0,15.0,177.0,...|         0|       0.0|
|[35.0,5.0,154.0,1...|         0|       0.0|
|[36.0,0.0,200.0,1...|         0|       0.0|
|[36.0,5.0,222.0,1...|         0|       0.0|
|[36.0,15.0,164.0,...|         0|       0.0|
|[36.0,20.0,159.0,...|         0|       0.0|
|[37.0,0.0,181.0,1...|         0|       0.0|
|[37.0,0.0,184.0,1...|         0|       0.0|
|[37.0,10.0,205.0,...|         0|       0.0|
|[37.0,15.0,275.0,...|         0|       0.0|
|[37.0,20.0,200.0,...|         0|       0.0|
|[37.0,